<a href="https://colab.research.google.com/github/pulkitv/test-rag/blob/main/Audio_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install openai==0.28
!pip install pyannote.audio
!pip install openai-whisper

!pip install torch==1.10.0+cu102
!pip install pyannote.audio==0.0.1

!pip install git+https://github.com/openai/whisper.git
!pip install gtts
!pip install pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 115.8 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.38.0
    Uninstalling google-auth-2.38.0:
      Successfully uninstalled google-auth-2.38.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.164.0
    Uninstalling google-api-python-client-2.164.0:
      Successfully uninstalled google-api-python-client-2.164.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.39.0 which is incompatible.


In [27]:
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID')
# ACTIVELOOP_TOKEN = userdata.get('ACTIVELOOP_TOKEN')
# LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
HUGGINGFACE_API_KEY = userdata.get('HUGGINGFACE_API_KEY')
GMAIL_PASSWORD = userdata.get('GMAIL_PASSWORD')


# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
# os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN
# os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["GMAIL_PASSWORD"] = GMAIL_PASSWORD

print("Done")

Done


In [21]:
from google.colab import files
import shutil

# Upload audio file
uploaded = files.upload()

# Move the uploaded file to the working directory
audio_file_path = list(uploaded.keys())[0]


Saving Fluent English conversation with buddy2.mp3 to Fluent English conversation with buddy2 (3).mp3


In [22]:
import whisper
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.core import Segment
import torch
import warnings
import logging

# Suppress FP16 warning on CPU
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU")

# Suppress PyTorch warnings and logs
logging.getLogger("torch").setLevel(logging.ERROR)

# Suppress SpeechBrain and pyannote logging
logging.getLogger("speechbrain").setLevel(logging.ERROR)
logging.getLogger("pyannote").setLevel(logging.ERROR)

# Load Whisper model for transcription
model = whisper.load_model("base")

# Perform transcription with Whisper and get word-level timestamps
result = model.transcribe(audio_file_path, word_timestamps=True)

# Get Whisper's word-level segments (start, end, and text)
segments = result['segments']  # This contains the start, end, and text of each segment

# Load pyannote's pre-trained speaker diarization model
pipeline = SpeakerDiarization.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token=HUGGINGFACE_API_KEY)

# Move pipeline to GPU if available
if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
    print("Pipeline is moved to GPU.")
else:
    print("GPU not available. Using CPU.")

# Perform speaker diarization on the audio
diarization = pipeline(audio_file_path)

# Create a list to store the formatted conversation
conversation = []

# Iterate through diarization and match with transcription segments
for turn, _, speaker in diarization.itertracks(yield_label=True):
    # Convert the start and end times to integer indices
    start_time = int(turn.start)
    end_time = int(turn.end)

    # Match the transcription segments by checking if the timestamps overlap with diarization times
    segment_text = ""
    for segment in segments:
        if segment['start'] >= start_time and segment['end'] <= end_time:
            segment_text += segment['text'] + " "

    # Append the formatted conversation
    conversation.append(f"Speaker {speaker}: {segment_text.strip()}")

# Print the formatted conversation
for entry in conversation:
    print(entry)


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in /root/.cache/torch/pyannote/speechbrain.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /root/.cache/torch/pyannote/speech

Pipeline is moved to GPU.


/usr/local/lib/python3.11/dist-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribute will be set to 0. If you are seeing this warning, please report by opening an issue on github (after checking for existing/closed ones). You may otherwise ignore this warning.
  warnings.warn(


Speaker SPEAKER_00: So this story I still remember of you that you said in your village initially people were not  treating you well like not giving you respect but after they came to know that you speak very good English  Everything has changed. They started giving you a huge respect and everything. I still remember that story actually  So actually I wanted no more on that like did you face only that issue or
Speaker SPEAKER_01: 
Speaker SPEAKER_01: First of all, I never wish start any conversation. I never ever demonstrate that oh, I can speak English  I always look for opportunities. You don't show off. Okay. What is the need?
Speaker SPEAKER_00: 
Speaker SPEAKER_00: 
Speaker SPEAKER_00: And but I am the opposite I show off all the time  Okay, look at my English
Speaker SPEAKER_01: 
Speaker SPEAKER_01: I never ever do that  I
Speaker SPEAKER_01: 
Speaker SPEAKER_01: If I feel yes, I need to take the call. I need to start the conversation. I certainly do that
Speaker SPEAKER_01: Okay

In [23]:
import openai

# Set your OpenAI API key
openai.api_key = os.environ["OPENAI_API_KEY"] # Replace 'your-openai-api-key' with your actual API key or better yet, store it as env variable

# Function to generate the summary using GPT-3.5 or GPT-4
def summarize_text(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use gpt-3.5-turbo instead of the deprecated text-davinci-003
        messages=[
            {"role": "system", "content": "You are a personal AI assistant who is hired to take notes and summarize meetings for future reference. Assume that the transcription provided is of a business meeting."},
            {"role": "user", "content": f"Summarize the following conversation (from an audio recording) without missing any important details. Segregate if there are multiple topics discussed. Each topic with their heading. List out any action items at the end separately for each speaker. Do not ask any follow up questions: {text}"}
        ],
        max_tokens=300
    )
    return response.choices[0].message['content'].strip()

# Assuming transcribed_text is the result from your transcription model (e.g., Whisper)
# transcribed_text = "Your transcribed text goes here" #Use the output from Whisper instead of this hardcoded value
# Assuming transcribed_text is already defined from your previous Whisper transcription step
# transcribed_text = result['text']

# Get the summary
summary = summarize_text(conversation)
print("Summary: ", summary)

Summary:  ### English Proficiency and Perception
- **Speaker SPEAKER_00** recalls a story where initially in their village, they were not treated well until it was discovered they spoke English well. This changed how they were perceived, gaining them respect.
- **Speaker SPEAKER_01** mentions they never show off their English skills, only using them when necessary. They dislike pretentious displays of fluency.

### Social Perceptions and Behavior
- **Speaker SPEAKER_00** shares an experience in a restaurant where speaking English led to assumptions about their status until they realized the waiter didn't understand English.
- **Speaker SPEAKER_01** emphasizes the importance of mutual understanding in communication, irrespective of language proficiency.

### Gender and Language Dynamics
- **Speaker SPEAKER_01** notes that some women may behave superior in language interactions when it's not necessary, potentially making others feel inadequate.
- **Speaker SPEAKER_00** recounts experienc

In [28]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os

# Function to send email
def send_email(conversation, summary):
    # Set up the server
    smtp_server = "smtp.gmail.com"  # Using Gmail's SMTP server
    smtp_port = 587  # Standard SMTP port for TLS
    sender_email = "vashishta.pulkit@gmail.com"  # Your Gmail email address
    receiver_email = "vashishta.pulkit@gmail.com"  # Receiver's email address
    email_password = GMAIL_PASSWORD  # Your Gmail App Password (generated from Google account)

    # Create the email content
    subject = "Meeting Summary and Conversation"
    body = f"""
    <h2>Conversation:</h2>
    <pre>{conversation}</pre>

    <h2>Summary:</h2>
    <pre>{summary}</pre>
    """

    # Create MIME message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the body to the email
    msg.attach(MIMEText(body, "html"))

    # Connect to the server and send the email
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, email_password)  # Log in to your email account
            server.sendmail(sender_email, receiver_email, msg.as_string())  # Send the email
            print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")

# Send the email
send_email(conversation, summary)


Email sent successfully!
